Following this tutorial: https://www.youtube.com/watch?v=LhbDMv3iA9s

In [15]:
import pennylane as qml
from pennylane import numpy as np
import dimod

import generate

In [16]:
def to_ising(H):
    n = H.shape[0] 
    J = np.zeros((n, n))
    h = np.zeros(n)

    for i in range(n):
        for j in range(n):
            if i != j:
                J[i, j] = 0.5 * H[i, j]
                J[j, i] = J[i, j]
        h[i] = 0.5 * np.sum(H[i, :])
    return J, h

# Función para crear el Hamiltoniano
def create_hamiltonian(h, J):
    num_qubits= J.shape[0]
    H = qml.Identity(wires=0)
    
    for i in range(num_qubits):
        if h[i] != 0:
            H += h[i] * qml.PauliZ(i)
    
    for i in range(num_qubits):
        for j in range(i + 1, num_qubits):
            if J[i, j] != 0:
                H += J[i, j] * qml.PauliZ(i) @ qml.PauliZ(j)
    
    return H

In [28]:
# Q, x = generate.generate_qubo_matrix(20,3)
# Q = np.array(
#     [
#         [-10.0, 19.7365809, 19.7365809, 5.42015853, 5.42015853],
#         [19.7365809, -10.0, 20.67626392, 0.17675796, 0.85604541],
#         [19.7365809, 20.67626392, -10.0, 0.85604541, 0.17675796],
#         [5.42015853, 0.17675796, 0.85604541, -10.0, 0.32306662],
#         [5.42015853, 0.85604541, 0.17675796, 0.32306662, -10.0],
#     ]
# )

bqm = dimod.generators.randint(10, vartype=dimod.BINARY, high=100, low=0)
Q = bqm.to_numpy_matrix()

J, h = to_ising(Q)
H = create_hamiltonian(h, J)

/var/folders/y9/h6266tlx7n9bd2_vpvfqrttns60r8r/T/ipykernel_14722/1722528007.py:13: DeprecationWarning: bqm.to_numpy_matrix() is deprecated since dimod 0.10.0 and will be removed in 0.12.0
  Q = bqm.to_numpy_matrix()


In [30]:
H

(
    I(0)
  + 253.0 * Z(0)
  + 224.5 * Z(1)
  + 210.0 * Z(2)
  + 190.0 * Z(3)
  + 90.5 * Z(4)
  + 154.5 * Z(5)
  + 111.0 * Z(6)
  + 50.5 * Z(7)
  + 43.0 * Z(8)
)

In [31]:
# Set up a quantum device
dev = qml.device('lightning.qubit', wires=H.wires)

In [32]:
# Quantum function to evaluate the QUBO problem
@qml.qnode(dev)
def circuit(params):
    total_qubits = H.num_wires
    for i in range(total_qubits):
        qml.RY(params[i], wires=H.wires[i])
        
        if(i < total_qubits-1):
            qml.CNOT(wires=[H.wires[i], H.wires[i+1]])
    return qml.expval(H)

# @qml.qnode(dev)
# def circuit(params):
#     for param, wire in zip(params, H.wires):
#         qml.RY(param, wires=wire)
#     return qml.expval(H)

In [33]:
circuit(np.zeros(H.num_wires))

/Users/targuelles/Library/Caches/pypoetry/virtualenvs/basic-qubo-2hUiBxcz-py3.10/lib/python3.10/site-packages/pennylane/devices/preprocess.py:240: UserWarning: Differentiating with respect to the input parameters of Sum is not supported with the adjoint differentiation method. Gradients are computed only with regards to the trainable parameters of the circuit.

 Mark the parameters of the measured observables as non-trainable to silence this warning.
  warnings.warn(


1328.0

In [34]:
# print(qml.draw(circuit, expansion_strategy='device')(params = np.zeros(H.num_wires)))

In [35]:
params = np.random.rand(H.num_wires)
# params = np.full(H.num_wires, 0.5)

In [36]:
opt = qml.AdagradOptimizer(stepsize=0.5)
epochs = 200

for epoch in range(epochs):
  params = opt.step(circuit, params)

circuit(params)

-1326.0

In [37]:
dev = qml.device('lightning.qubit', wires=H.wires, shots=10)
@qml.qnode(dev)
def circuit(params):
    total_qubits = len(H.wires)
    for i in range(total_qubits):
        qml.RY(params[i], wires=H.wires[i])
        
        if(i < total_qubits-1):
            qml.CNOT(wires=[H.wires[i], H.wires[i+1]])
    return qml.sample()

# dev = qml.device('default.qubit', wires=H.wires, shots=1)
# @qml.qnode(dev)
# def circuit(params):
#     for param, wire in zip(params, H.wires):
#         qml.RY(param, wires=wire)
#     return qml.sample()

In [38]:
circuit(params), H.wires

(array([[1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1]]),
 <Wires = [0, 1, 2, 3, 4, 5, 6, 7, 8]>)

In [39]:
# print('Posible solucion optima:' + str(x))